# numpy.random

## Overview

## Simple random data

## Permutations

## Some distribution functions

### 1. The Gumbel Distribution
#### `random.gumbel`
$$ p(x)=\frac{e^{-(x-\mu)/\beta}}{\beta}e^{-e^{-(x-\mu)/\beta}} $$

### 2.

### 3.

### 4.

### 5.

## Seeds